In [1]:
"""
EDA-Desk PRO HYBRID — Version finale complète
Architecture à 4 zones + Accumulation des résultats + Statistiques catégorielles
Version améliorée : Support Excel + Zoom + Headers stylisés
"""

import tkinter as tk
from tkinter import filedialog, messagebox
import ttkbootstrap as ttk
from ttkbootstrap.constants import *
from ttkbootstrap.widgets import ToastNotification
from ttkbootstrap.dialogs import Messagebox
import pandas as pd
import numpy as np
from typing import Optional, Dict, List, Tuple
from scipy import stats
import openpyxl  # Pour support Excel
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import seaborn as sns
from datetime import datetime
import sqlite3
import os
import csv

# Imports pour exports

from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak, Image as RLImage
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from datetime import datetime
from typing import Dict

class HistoryManager:
    """Gestionnaire d'historique avec SQLite"""
    
    def __init__(self, db_path="eda_history.db"):
        self.db_path = db_path
        self._init_database()
    
    def _init_database(self):
        """Initialiser la base de données"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS analysis_history (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                filename TEXT NOT NULL,
                filepath TEXT,
                loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                rows INTEGER,
                columns INTEGER,
                numeric_vars INTEGER,
                categorical_vars INTEGER,
                boolean_vars INTEGER,
                quality_score REAL,
                missing_pct REAL,
                outliers_count INTEGER,
                notes TEXT
            )
        ''')
        
        conn.commit()
        conn.close()
    
    def add_entry(self, data_info: Dict):
        """Ajouter une entrée à l'historique"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT INTO analysis_history 
            (filename, filepath, rows, columns, numeric_vars, categorical_vars, 
             boolean_vars, quality_score, missing_pct, outliers_count, notes)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            data_info.get('filename'),
            data_info.get('filepath'),
            data_info.get('rows'),
            data_info.get('columns'),
            data_info.get('numeric_vars'),
            data_info.get('categorical_vars'),
            data_info.get('boolean_vars'),
            data_info.get('quality_score'),
            data_info.get('missing_pct'),
            data_info.get('outliers_count'),
            data_info.get('notes', '')
        ))
        
        conn.commit()
        conn.close()
    
    def get_history(self, limit=50):
        """Récupérer l'historique"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            SELECT * FROM analysis_history 
            ORDER BY loaded_at DESC 
            LIMIT ?
        ''', (limit,))
        
        columns = [desc[0] for desc in cursor.description]
        results = [dict(zip(columns, row)) for row in cursor.fetchall()]
        
        conn.close()
        return results



from export_templates_masterclass import export_to_word_masterclass, export_to_pdf_masterclass

class ReportExporter:
    @staticmethod
    def export_to_word(data_info, stats, output_path, accumulated_content=None):
        export_to_word_masterclass(data_info, stats, output_path, accumulated_content)
    
    @staticmethod
    def export_to_pdf(data_info, stats, output_path, accumulated_content=None):
        export_to_pdf_masterclass(data_info, stats, output_path, accumulated_content)



 
class EDADeskHybrid:
    """Application EDA-Desk PRO Hybrid - Version finale complète"""
    
    def __init__(self, root):
        self.root = root
        self.data: Optional[pd.DataFrame] = None
        self.filename: str = ""
        self.filepath: str = ""
        self.variable_types: Dict[str, str] = {}
        self.numeric_vars: List[str] = []
        self.categorical_vars: List[str] = []
        self.boolean_vars: List[str] = []
        
        # Résultats
        self.missing_values: Dict[str, Tuple[int, float]] = {}
        self.high_missing_vars: List[str] = []
        self.quasi_constant_vars: List[str] = []
        self.outliers_info: Dict[str, Dict] = {}
        self.current_stats: Dict = {}
        self.last_analysis_type: str = ""
        self.last_analysis_report: str = ""
        
        # Accumulation des résultats
        self.accumulated_reports: List[Dict] = []
        
        # Zoom factor
        self.zoom_factor: float = 1.0
        
        # Managers
        self.history_manager = HistoryManager()
        self.report_exporter = ReportExporter()
        
        # Zoom factor
        self.zoom_factor: float = 1.0
        self.viz_zoom_factor: float = 1.0  # AJOUT pour zoom visualisations
        self.current_fig = None  # AJOUT pour stocker la figure actuelle
        
        
        # Configuration matplotlib
        sns.set_style("whitegrid")
        plt.rcParams['figure.facecolor'] = 'white'
        
        self._setup_window()
        self._create_ui()
        
    def _setup_window(self):
        """Configuration de la fenêtre avec adaptation automatique"""
        self.root.title("EDA Exploratory Data Analysis Desk")
        
        # Obtenir les dimensions de l'écran
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        
        # Utiliser 85% de la taille de l'écran comme BASE
        self.base_width = int(screen_width * 0.85)
        self.base_height = int(screen_height * 0.85)
        
        # Centrer
        x = (screen_width - self.base_width) // 2
        y = (screen_height - self.base_height) // 2
        
        self.root.geometry(f"{self.base_width}x{self.base_height}+{x}+{y}")
        self.root.minsize(1000, 700)
    
    
    
    def _create_ui(self):
        """Création de l'interface"""
        # Menu bar
        self._create_menubar()
        
        # Notebook principal
        self.notebook = ttk.Notebook(self.root, bootstyle="primary")
        self.notebook.pack(fill=BOTH, expand=YES, padx=10, pady=10)
        
        # Onglet Vue d'ensemble (avec scroll)
        tab_overview_container = ttk.Frame(self.notebook)
        self.notebook.add(tab_overview_container, text="Vue d'ensemble")
        self.tab_overview = self._create_scrollable_frame(tab_overview_container)
        
        # Onglet Résultats (avec scroll)
        tab_results_container = ttk.Frame(self.notebook)
        self.notebook.add(tab_results_container, text="Résultats & Exports")
        self.tab_results = self._create_scrollable_frame(tab_results_container)
        
        # Onglet Statistiques (avec scroll)
        tab_stats_container = ttk.Frame(self.notebook)
        self.notebook.add(tab_stats_container, text="Statistiques avancées")
        self.tab_stats = self._create_scrollable_frame(tab_stats_container)
        
        # Onglet Visualisations (avec scroll)
        
        self.tab_viz = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_viz, text="Visualisations")
        
        # Onglet Données (avec scroll)
        
        self.tab_data = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_data, text="Données complètes")
        
        # Onglet Historique (avec scroll)
        tab_history_container = ttk.Frame(self.notebook)
        self.notebook.add(tab_history_container, text="Historique")
        self.tab_history = self._create_scrollable_frame(tab_history_container)
        
        # Créer contenu
        self._create_overview_tab_4_zones()
        self._create_results_tab()
        self._create_stats_tab()
        self._create_viz_tab()
        self._create_data_tab()
        self._create_history_tab()

    def _create_scrollable_frame(self, parent):
        """Créer un frame scrollable pour un onglet"""
        # Container principal
        container = ttk.Frame(parent)
        container.pack(fill=BOTH, expand=YES)
        
        # Canvas
        canvas = tk.Canvas(container, highlightthickness=0, bg='#f8f9fa')
        
        # Scrollbar verticale
        v_scrollbar = ttk.Scrollbar(container, orient=VERTICAL, command=canvas.yview, bootstyle="primary-round")
        v_scrollbar.pack(side=RIGHT, fill=Y)
        
        # Scrollbar horizontale
        h_scrollbar = ttk.Scrollbar(container, orient=HORIZONTAL, command=canvas.xview, bootstyle="primary-round")
        h_scrollbar.pack(side=BOTTOM, fill=X)
        
        # Pack canvas
        canvas.pack(side=LEFT, fill=BOTH, expand=YES)
        
        # Configuration scrollbars
        canvas.configure(yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)
        
        # Frame interne scrollable
        scrollable_frame = ttk.Frame(canvas)
        canvas_window = canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        
        # Mise à jour de la région scrollable
        def configure_scroll_region(event=None):
            canvas.configure(scrollregion=canvas.bbox("all"))
        
        scrollable_frame.bind("<Configure>", configure_scroll_region)
        
        # Adapter la largeur du frame au canvas
        def configure_canvas_window(event):
            canvas.itemconfig(canvas_window, width=event.width)
        
        canvas.bind("<Configure>", configure_canvas_window)
        
        # Scroll avec la molette de souris
        def on_mousewheel(event):
            canvas.yview_scroll(int(-1*(event.delta/120)), "units")
        
        def bind_mousewheel(event):
            canvas.bind_all("<MouseWheel>", on_mousewheel)
        
        def unbind_mousewheel(event):
            canvas.unbind_all("<MouseWheel>")
        
        # Activer le scroll uniquement quand la souris est sur cet onglet
        canvas.bind("<Enter>", bind_mousewheel)
        canvas.bind("<Leave>", unbind_mousewheel)
        
        return scrollable_frame
        
    def _create_menubar(self):
        """Créer la barre de menu"""
        menubar = tk.Menu(self.root)
        self.root.config(menu=menubar)
        
        # Menu Fichier
        file_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Fichier", menu=file_menu)
        file_menu.add_command(label="Ouvrir CSV...", command=self._open_file, accelerator="Ctrl+O")
        file_menu.add_command(label="Ouvrir Excel...", command=self._open_excel_file, accelerator="Ctrl+E")
        file_menu.add_separator()
        file_menu.add_command(label="Quitter", command=self.root.quit, accelerator="Ctrl+Q")
        
        # Menu Export
        export_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Export", menu=export_menu)
        export_menu.add_command(label="Export Word (.docx)", command=self._export_word)
        export_menu.add_command(label="Export PDF", command=self._export_pdf)
        export_menu.add_command(label="Export Excel (.xlsx)", command=self._export_excel)
        
        # Menu Affichage
        view_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Affichage", menu=view_menu)
        view_menu.add_command(label="Zoom +", command=self._zoom_in, accelerator="Ctrl++")
        view_menu.add_command(label="Zoom -", command=self._zoom_out, accelerator="Ctrl+-")
        view_menu.add_command(label="Zoom 100%", command=self._zoom_reset, accelerator="Ctrl+0")
        
        # Menu Navigation
        nav_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Navigation", menu=nav_menu)
        nav_menu.add_command(label="Vue d'ensemble", command=lambda: self.notebook.select(0))
        nav_menu.add_command(label="Résultats & Exports", command=lambda: self.notebook.select(1))
        nav_menu.add_command(label="Statistiques", command=lambda: self.notebook.select(2))
        nav_menu.add_command(label="Visualisations", command=lambda: self.notebook.select(3))
        nav_menu.add_command(label="Données", command=lambda: self.notebook.select(4))
        nav_menu.add_command(label="Historique", command=lambda: self.notebook.select(5))
        
        # Menu Aide
        help_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Aide", menu=help_menu)
        help_menu.add_command(label="À propos", command=self._show_about)
        
        # Raccourcis
        self.root.bind('<Control-o>', lambda e: self._open_file())
        self.root.bind('<Control-e>', lambda e: self._open_excel_file())
        self.root.bind('<Control-q>', lambda e: self.root.quit())
        self.root.bind('<Control-plus>', lambda e: self._zoom_in())
        self.root.bind('<Control-minus>', lambda e: self._zoom_out())
        self.root.bind('<Control-0>', lambda e: self._zoom_reset())
    
    def _create_styled_header(self, parent, title, subtitle):
        """Créer un header stylisé avec gradient bleu clair"""
        header_container = ttk.Frame(parent)
        header_container.pack(fill=X, padx=0, pady=0)
        
        title_canvas = tk.Canvas(header_container, height=120, highlightthickness=0)
        title_canvas.pack(fill=X)
        
            # Gradient bleu clair élégant
        for i in range(120):
                r = int(74 + (100 - 74) * i / 120)      # 4A -> 64
                g = int(144 + (180 - 144) * i / 120)    # 90 -> B4
                b = int(226 + (240 - 226) * i / 120)    # E2 -> F0
                color = f'#{r:02x}{g:02x}{b:02x}'
                title_canvas.create_line(0, i, 2000, i, fill=color)
                    
        title_canvas.create_text(
            750, 40,
            text=title,
            font=("Segoe UI", 28, "bold"),
            fill='white'
        )
        
        title_canvas.create_text(
            750, 80,
            text=subtitle,
            font=("Segoe UI", 11),
            fill='#e0e7ff'
        )
        
        return header_container
    
    # ============================================================
    # ZOOM FUNCTIONS
    # ============================================================
    
    def _zoom_in(self):
        """Augmenter le zoom"""
        self.zoom_factor = min(2.0, self.zoom_factor + 0.1)
        self._apply_zoom()
        ToastNotification(
            title="Zoom",
            message=f"Zoom: {int(self.zoom_factor * 100)}%",
            duration=1500,
            bootstyle="info"
        ).show_toast()
    
    def _zoom_out(self):
        """Diminuer le zoom"""
        self.zoom_factor = max(0.5, self.zoom_factor - 0.1)
        self._apply_zoom()
        ToastNotification(
            title="Zoom",
            message=f"Zoom: {int(self.zoom_factor * 100)}%",
            duration=1500,
            bootstyle="info"
        ).show_toast()
    
    def _zoom_reset(self):
        """Réinitialiser le zoom"""
        self.zoom_factor = 1.0
        self._apply_zoom()
        ToastNotification(
            title="Zoom",
            message="Zoom: 100%",
            duration=1500,
            bootstyle="info"
        ).show_toast()
    
    def _apply_zoom(self):
        """Appliquer le facteur de zoom"""
        # Calculer les nouvelles dimensions basées sur la taille de base adaptative
        new_width = int(self.base_width * self.zoom_factor)
        new_height = int(self.base_height * self.zoom_factor)
        
        # Centrer la fenêtre
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        x = (screen_width - new_width) // 2
        y = (screen_height - new_height) // 2
        
        self.root.geometry(f"{new_width}x{new_height}+{x}+{y}")
    # ============================================================
    # ONGLET 1: VUE D'ENSEMBLE
    # ============================================================
    
    def _create_overview_tab_4_zones(self):
        """Onglet Vue d'ensemble - Architecture 4 zones"""
        
        # Header stylisé
        self._create_styled_header(
            self.tab_overview,
            "VUE D'ENSEMBLE",
            "Chargement et aperçu des données"
        )
        
        # ZONE 1
        self._create_zone1_top_bar()
        
        # Container central
        central_container = ttk.Frame(self.tab_overview)
        central_container.pack(fill=BOTH, expand=True, padx=10, pady=10)
        
        # ZONE 2 et 3
        self._create_zone2_left_data_preview(central_container)
        self._create_zone3_right_controls(central_container)
        
        # ZONE 4
        self._create_zone4_bottom_results()
    
    def _create_zone1_top_bar(self):
        """ZONE 1: Barre supérieure"""
        top_card = ttk.Labelframe(
            self.tab_overview,
            text="Sélection de fichier",
            bootstyle="primary",
            padding=15
        )
        top_card.pack(fill=X, padx=10, pady=(10, 5))
        
        content = ttk.Frame(top_card)
        content.pack(fill=X)
        
        # Boutons
        left_frame = ttk.Frame(content)
        left_frame.pack(side=LEFT, fill=Y)
        
        ttk.Button(
            left_frame,
            text="Ouvrir CSV",
            command=self._open_file,
            bootstyle="success",
            width=20
        ).pack(pady=5, side=LEFT, padx=5)
        
        ttk.Button(
            left_frame,
            text="Ouvrir Excel",
            command=self._open_excel_file,
            bootstyle="success",
            width=20
        ).pack(pady=5, side=LEFT, padx=5)
        
        # Séparateur
        ttk.Separator(content, orient=VERTICAL, bootstyle="secondary").pack(side=LEFT, fill=Y, padx=20)
        
        # Infos
        right_frame = ttk.Frame(content)
        right_frame.pack(side=LEFT, fill=BOTH, expand=YES)
        
        info_grid = ttk.Frame(right_frame)
        info_grid.pack(fill=BOTH, expand=YES)
        
        # Nom fichier
        file_frame = ttk.Frame(info_grid)
        file_frame.pack(fill=X, pady=5)
        
        ttk.Label(
            file_frame,
            text="Fichier chargé:",
            bootstyle="secondary",
            font=("Segoe UI", 9)
        ).pack(side=LEFT, padx=(0, 10))
        
        self.lbl_filename = ttk.Label(
            file_frame,
            text="Aucun fichier",
            font=("Segoe UI", 11, "bold"),
            bootstyle="primary"
        )
        self.lbl_filename.pack(side=LEFT)
        
        # Dimensions
        dim_frame = ttk.Frame(info_grid)
        dim_frame.pack(fill=X, pady=5)
        
        ttk.Label(
            dim_frame,
            text="Dimensions:",
            bootstyle="secondary",
            font=("Segoe UI", 9)
        ).pack(side=LEFT, padx=(0, 10))
        
        self.lbl_dimensions = ttk.Label(
            dim_frame,
            text="0 lignes × 0 colonnes",
            font=("Segoe UI", 11, "bold"),
            bootstyle="info"
        )
        self.lbl_dimensions.pack(side=LEFT)
    
    def _create_zone2_left_data_preview(self, parent):
        """ZONE 2: Aperçu des données"""
        left_card = ttk.Labelframe(
            parent,
            text="Aperçu des données (50 premières lignes)",
            bootstyle="info",
            padding=10
        )
        left_card.pack(side=LEFT, fill=BOTH, expand=YES, padx=(0, 5))
        
        scroll_container = ttk.Frame(left_card)
        scroll_container.pack(fill=BOTH, expand=YES)
        
        vsb = ttk.Scrollbar(scroll_container, orient=VERTICAL, bootstyle="info-round")
        vsb.pack(side=RIGHT, fill=Y)
        
        hsb = ttk.Scrollbar(scroll_container, orient=HORIZONTAL, bootstyle="info-round")
        hsb.pack(side=BOTTOM, fill=X)
        
        self.data_preview = tk.Text(
            scroll_container,
            wrap=NONE,
            font=("Consolas", 9),
            yscrollcommand=vsb.set,
            xscrollcommand=hsb.set,
            relief=FLAT,
            bg='#f8f9fa',
            fg='#212529',
            padx=10,
            pady=10
        )
        self.data_preview.pack(fill=BOTH, expand=YES)
        
        vsb.config(command=self.data_preview.yview)
        hsb.config(command=self.data_preview.xview)
        
        welcome = """
                  ZONE 2 - APERÇU DES DONNÉES
                  
Chargez un fichier CSV ou Excel pour visualiser les données.
"""
        self.data_preview.insert('1.0', welcome)
        self.data_preview.config(state=DISABLED)
    
    def _create_zone3_right_controls(self, parent):
        """ZONE 3: Analyses et contrôles"""
        right_card = ttk.Labelframe(
            parent,
            text="Analyses et contrôles",
            bootstyle="warning",
            padding=10
        )
        right_card.pack(side=RIGHT, fill=BOTH, padx=(5, 0))
        right_card.configure(width=400)
        
        canvas = tk.Canvas(right_card, bg='#f8f9fa', highlightthickness=0)
        scrollbar = ttk.Scrollbar(right_card, orient=VERTICAL, command=canvas.yview, bootstyle="warning-round")
        
        controls_frame = ttk.Frame(canvas)
        controls_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=controls_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        scrollbar.pack(side=RIGHT, fill=Y)
        canvas.pack(side=LEFT, fill=BOTH, expand=YES)
        
        # Sélection variables
        select_section = ttk.Labelframe(
            controls_frame,
            text="Sélection de variables",
            bootstyle="info",
            padding=15
        )
        select_section.pack(fill=X, padx=10, pady=10)
        
        var1_frame = ttk.Frame(select_section)
        var1_frame.pack(fill=X, pady=5)
        ttk.Label(var1_frame, text="Variable numérique 1:", font=("Segoe UI", 9)).pack(anchor=W)
        self.var1_combo = ttk.Combobox(var1_frame, state='readonly', width=30)
        self.var1_combo.pack(fill=X, pady=(5, 0))
        
        var2_frame = ttk.Frame(select_section)
        var2_frame.pack(fill=X, pady=5)
        ttk.Label(var2_frame, text="Variable numérique 2:", font=("Segoe UI", 9)).pack(anchor=W)
        self.var2_combo = ttk.Combobox(var2_frame, state='readonly', width=30)
        self.var2_combo.pack(fill=X, pady=(5, 0))
        
        cat_frame = ttk.Frame(select_section)
        cat_frame.pack(fill=X, pady=5)
        ttk.Label(cat_frame, text="Variable catégorielle:", font=("Segoe UI", 9)).pack(anchor=W)
        self.cat_combo = ttk.Combobox(cat_frame, state='readonly', width=30)
        self.cat_combo.pack(fill=X, pady=(5, 0))
        
        ttk.Button(
            select_section,
            text="Calculer statistiques détaillées",
            command=self._go_to_stats,
            bootstyle="success",
            width=35
        ).pack(pady=10)
        
        # Analyses statistiques
        analysis_section = ttk.Labelframe(
            controls_frame,
            text="Analyses statistiques",
            bootstyle="primary",
            padding=15
        )
        analysis_section.pack(fill=X, padx=10, pady=10)
        
        ttk.Button(
            analysis_section,
            text="Types de variables",
            command=self._show_detailed_types,
            bootstyle="info",
            width=35
        ).pack(pady=5)
        
        ttk.Button(
            analysis_section,
            text="Valeurs manquantes",
            command=self._analyze_missing_values,
            bootstyle="warning",
            width=35
        ).pack(pady=5)
        
        ttk.Button(
            analysis_section,
            text="Variables constantes",
            command=self._detect_quasi_constant,
            bootstyle="secondary",
            width=35
        ).pack(pady=5)
        
        ttk.Button(
            analysis_section,
            text="Outliers (IQR)",
            command=self._detect_outliers,
            bootstyle="danger",
            width=35
        ).pack(pady=5)
        
        ttk.Separator(analysis_section, bootstyle="primary").pack(fill=X, pady=10)
        
        ttk.Button(
            analysis_section,
            text="RAPPORT COMPLET",
            command=self._full_quality_report,
            bootstyle="success",
            width=35
        ).pack(pady=5)
        
        # Visualisations
        viz_section = ttk.Labelframe(
            controls_frame,
            text="Visualisations",
            bootstyle="secondary",
            padding=15
        )
        viz_section.pack(fill=X, padx=10, pady=10)
        
        ttk.Button(
            viz_section,
            text="Ouvrir l'onglet Visualisations",
            command=lambda: self.notebook.select(3),
            bootstyle="info-outline",
            width=35
        ).pack(pady=5)
    
    def _create_zone4_bottom_results(self):
        """ZONE 4: Résultats rapides"""
        bottom_card = ttk.Labelframe(
            self.tab_overview,
            text=" Aperçu rapide des résultats",
            bootstyle="success",
            padding=10
        )
        bottom_card.pack(fill=BOTH, padx=10, pady=(5, 10))
        bottom_card.configure(height=250)
        
        header = ttk.Frame(bottom_card)
        header.pack(fill=X, pady=(0, 10))
        
        ttk.Label(
            header,
            text="Aperçu rapide",
            font=("Segoe UI", 12, "bold")
        ).pack(side=LEFT)
        
        self.status_badge = ttk.Label(
            header,
            text="Prêt",
            bootstyle="success",
            font=("Segoe UI", 9)
        )
        self.status_badge.pack(side=RIGHT, padx=10)
        
        ttk.Button(
            header,
            text="Voir les résultats détaillés",
            command=lambda: self.notebook.select(1),
            bootstyle="primary-outline"
        ).pack(side=RIGHT)
        
        results_container = ttk.Frame(bottom_card)
        results_container.pack(fill=BOTH, expand=YES)
        
        scrollbar = ttk.Scrollbar(results_container, bootstyle="success-round")
        scrollbar.pack(side=RIGHT, fill=Y)
        
        self.results_text = tk.Text(
            results_container,
            wrap=WORD,
            font=("Consolas", 9),
            yscrollcommand=scrollbar.set,
            relief=FLAT,
            bg='#f8f9fa',
            fg='#212529',
            padx=15,
            pady=15
        )
        self.results_text.pack(fill=BOTH, expand=YES)
        scrollbar.config(command=self.results_text.yview)
        
        welcome = """
                      ZONE 4 - APERÇU RAPIDE DES RÉSULTATS

Lancez des analyses pour voir les résultats ici.
Les résultats complets s'accumulent dans l'onglet "Résultats & Exports".
"""
        self.results_text.insert('1.0', welcome)
        self.results_text.config(state=DISABLED)
    
    # ============================================================
    # ONGLET 2: RÉSULTATS & EXPORTS
    # ============================================================
    
    def _create_results_tab(self):
        """Onglet Résultats avec accumulation"""
        
        # Header stylisé
        self._create_styled_header(
            self.tab_results,
            "RÉSULTATS D'ANALYSE",
            "Accumulation progressive - Exports professionnels - Partage facile"
        )
        
        # Container principal
        main_container = ttk.Frame(self.tab_results)
        main_container.pack(fill=BOTH, expand=YES, padx=20, pady=20)
        
        # Colonne gauche
        left_column = ttk.Frame(main_container)
        left_column.pack(side=LEFT, fill=BOTH, expand=YES, padx=(0, 10))
        
        info_card = ttk.Labelframe(
            left_column,
            text="Informations du fichier analysé",
            bootstyle="info",
            padding=20
        )
        info_card.pack(fill=X, pady=(0, 15))
        
        info_grid = ttk.Frame(info_card)
        info_grid.pack(fill=X)
        
        self.result_filename_lbl = self._create_info_row(info_grid, "Fichier:", "Aucun fichier", 0)
        self.result_dimensions_lbl = self._create_info_row(info_grid, "Dimensions:", "—", 1)
        self.result_types_lbl = self._create_info_row(info_grid, "Types:", "—", 2)
        self.result_quality_lbl = self._create_info_row(info_grid, "Qualité:", "—", 3)
        
        results_card = ttk.Labelframe(
            left_column,
            text="Résultats accumulés",
            bootstyle="success",
            padding=15
        )
        results_card.pack(fill=BOTH, expand=YES)
        
        results_scroll_frame = ttk.Frame(results_card)
        results_scroll_frame.pack(fill=BOTH, expand=YES)
        
        results_scrollbar = ttk.Scrollbar(results_scroll_frame, bootstyle="success-round")
        results_scrollbar.pack(side=RIGHT, fill=Y)
        
        self.results_detail_text = tk.Text(
            results_scroll_frame,
            wrap=WORD,
            font=("Consolas", 10),
            yscrollcommand=results_scrollbar.set,
            relief=FLAT,
            bg='#ffffff',
            fg='#1e293b',
            padx=20,
            pady=20
        )
        self.results_detail_text.pack(fill=BOTH, expand=YES)
        results_scrollbar.config(command=self.results_detail_text.yview)
        
        default_msg = """
                    ESPACE RÉSULTATS ACCUMULÉS

Lancez des analyses depuis l'onglet "Vue d'ensemble"
   Chaque analyse s'ajoutera automatiquement ici

Avantages:
   - Toutes vos analyses dans un seul rapport
   - Export facile en Word, PDF ou Excel
   - Construction progressive de votre rapport
"""
        self.results_detail_text.insert('1.0', default_msg)
        self.results_detail_text.config(state=DISABLED)
        
        # Colonne droite
        right_column = ttk.Frame(main_container)
        right_column.pack(side=RIGHT, fill=Y, padx=(10, 0))
        right_column.configure(width=350)
        
        status_card = ttk.Labelframe(
            right_column,
            text="Statut",
            bootstyle="warning",
            padding=20
        )
        status_card.pack(fill=X, pady=(0, 15))
        
        self.result_status_label = ttk.Label(
            status_card,
            text="En attente...",
            font=("Segoe UI", 11),
            bootstyle="secondary",
            wraplength=280
        )
        self.result_status_label.pack(pady=10)
        
        self.result_analysis_type_label = ttk.Label(
            status_card,
            text="Analyses: 0",
            font=("Segoe UI", 9),
            bootstyle="secondary"
        )
        self.result_analysis_type_label.pack()
        
        ttk.Separator(right_column, bootstyle="secondary").pack(fill=X, pady=15)
        
        export_card = ttk.Labelframe(
            right_column,
            text="Exporter",
            bootstyle="primary",
            padding=20
        )
        export_card.pack(fill=X, pady=(0, 15))
        
        ttk.Label(
            export_card,
            text="Choisissez votre format:",
            font=("Segoe UI", 11, "bold"),
            bootstyle="primary"
        ).pack(pady=(0, 15))
        
        word_frame = ttk.Frame(export_card)
        word_frame.pack(fill=X, pady=10)
        ttk.Button(
            word_frame,
            text="Export Word (.docx)",
            command=self._export_word,
            bootstyle="info",
            width=30
        ).pack()
        ttk.Label(word_frame, text="Format professionnel", font=("Segoe UI", 9), bootstyle="secondary").pack(pady=(3, 0))
        
        pdf_frame = ttk.Frame(export_card)
        pdf_frame.pack(fill=X, pady=10)
        ttk.Button(
            pdf_frame,
            text="Export PDF",
            command=self._export_pdf,
            bootstyle="danger",
            width=30
        ).pack()
        ttk.Label(pdf_frame, text="Format universel", font=("Segoe UI", 9), bootstyle="secondary").pack(pady=(3, 0))
        
        excel_frame = ttk.Frame(export_card)
        excel_frame.pack(fill=X, pady=10)
        ttk.Button(
            excel_frame,
            text="Export Excel (.xlsx)",
            command=self._export_excel,
            bootstyle="success",
            width=30
        ).pack()
        ttk.Label(excel_frame, text="Données + statistiques", font=("Segoe UI", 9), bootstyle="secondary").pack(pady=(3, 0))
        
        ttk.Separator(right_column, bootstyle="secondary").pack(fill=X, pady=15)
        
        actions_card = ttk.Labelframe(
            right_column,
            text="Actions",
            bootstyle="info",
            padding=20
        )
        actions_card.pack(fill=X)
        
        ttk.Button(
            actions_card,
            text="Actualiser",
            command=self._refresh_results_tab,
            bootstyle="info",
            width=30
        ).pack(pady=5)
        
        ttk.Button(
            actions_card,
            text="Effacer tout",
            command=self._clear_accumulated_results,
            bootstyle="danger",
            width=30
        ).pack(pady=5)
        
        ttk.Button(
            actions_card,
            text="Vue d'ensemble",
            command=lambda: self.notebook.select(0),
            bootstyle="secondary",
            width=30
        ).pack(pady=5)
    
    def _create_info_row(self, parent, label_text, value_text, row):
        """Créer ligne d'info"""
        row_frame = ttk.Frame(parent)
        row_frame.grid(row=row, column=0, sticky=W, pady=8)
        
        ttk.Label(
            row_frame,
            text=label_text,
            font=("Segoe UI", 10, "bold"),
            bootstyle="info",
            width=15
        ).pack(side=LEFT, anchor=W)
        
        value_label = ttk.Label(
            row_frame,
            text=value_text,
            font=("Segoe UI", 10, "bold"),
            bootstyle="primary"
        )
        value_label.pack(side=LEFT, anchor=W, padx=10)
        
        return value_label
    
    # ============================================================
    # ACCUMULATION DES RÉSULTATS
    # ============================================================
    
    def _add_analysis_to_accumulator(self, analysis_type: str, report: str):
        """Ajouter analyse à l'accumulateur"""
        existing_index = None
        for i, item in enumerate(self.accumulated_reports):
            if item['type'] == analysis_type:
                existing_index = i
                break
        
        entry = {
            'type': analysis_type,
            'report': report,
            'timestamp': datetime.now().strftime('%H:%M:%S')
        }
        
        if existing_index is not None:
            self.accumulated_reports[existing_index] = entry
        else:
            self.accumulated_reports.append(entry)
        
        self._update_accumulated_results()
    
    def _update_accumulated_results(self):
        """Mettre à jour onglet avec résultats accumulés"""
        if not self.accumulated_reports:
            return
        
        full_report = f"""
                         RAPPORT D'ANALYSE COMPLET
                         {len(self.accumulated_reports)} analyse(s) effectuée(s)

Fichier : {self.filename}
Date : {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}

"""
        
        for i, entry in enumerate(self.accumulated_reports, 1):
            full_report += f"\n\n{'=' * 85}\n"
            full_report += f"ANALYSE #{i} - {entry['type'].upper()} (à {entry['timestamp']})\n"
            full_report += f"{'=' * 85}\n\n"
            full_report += entry['report']
        
        full_report += f"\n\n{'=' * 85}\n"
        full_report += "Toutes les analyses sont disponibles pour l'export.\n"
        full_report += "Utilisez les boutons d'export (Word, PDF, Excel).\n"
        full_report += f"{'=' * 85}\n"
        
        self.results_detail_text.config(state=NORMAL)
        self.results_detail_text.delete('1.0', END)
        self.results_detail_text.insert('1.0', full_report)
        self.results_detail_text.config(state=DISABLED)
        
        self.result_status_label.config(
            text=f"{len(self.accumulated_reports)} analyse(s)",
            bootstyle="success"
        )
        self.result_analysis_type_label.config(
            text=f"Analyses: {len(self.accumulated_reports)}"
        )
        
        self.last_analysis_report = full_report
    
    def _clear_accumulated_results(self):
        """Effacer résultats accumulés"""
        if not self.accumulated_reports:
            messagebox.showinfo("Info", "Aucun résultat à effacer")
            return
        
        if Messagebox.yesno(
            "Confirmation",
            f"Effacer les {len(self.accumulated_reports)} analyse(s) ?"
        ):
            self.accumulated_reports = []
            
            self.results_detail_text.config(state=NORMAL)
            self.results_detail_text.delete('1.0', END)
            default_msg = """
                    RÉSULTATS EFFACÉS

Lancez de nouvelles analyses depuis "Vue d'ensemble".
"""
            self.results_detail_text.insert('1.0', default_msg)
            self.results_detail_text.config(state=DISABLED)
            
            self.result_status_label.config(
                text="En attente...",
                bootstyle="secondary"
            )
            
            ToastNotification(
                title="Effacé",
                message="Résultats supprimés",
                duration=2000,
                bootstyle="info"
            ).show_toast()
    
    def _refresh_results_tab(self):
        """Actualiser"""
        if self.data is None:
            messagebox.showinfo("Info", "Aucun fichier chargé")
            return
        
        self._update_results_tab_info()
        
        ToastNotification(
            title="Actualisé",
            message="Résultats mis à jour",
            duration=2000,
            bootstyle="success"
        ).show_toast()
    
    def _update_results_tab_info(self):
        """MAJ infos onglet résultats"""
        if self.data is None:
            return
        
        self.result_filename_lbl.config(text=self.filename)
        self.result_dimensions_lbl.config(
            text=f"{self.data.shape[0]:,} × {self.data.shape[1]}"
        )
        self.result_types_lbl.config(
            text=f"N:{len(self.numeric_vars)} C:{len(self.categorical_vars)} B:{len(self.boolean_vars)}"
        )
        
        quality_score = self._calculate_quality_score()
        if quality_score >= 90:
            grade = "EXCELLENT"
            style = "success"
        elif quality_score >= 75:
            grade = "BON"
            style = "warning"
        elif quality_score >= 60:
            grade = "MOYEN"
            style = "warning"
        else:
            grade = "FAIBLE"
            style = "danger"
        
        self.result_quality_lbl.config(
            text=f"{quality_score:.1f}/100 {grade}",
            bootstyle=style
        )
    
    # ============================================================
    # ONGLET 3: STATISTIQUES
    # ============================================================
    
    def _create_stats_tab(self):
        """Onglet Statistiques"""
        
        # Header stylisé
        self._create_styled_header(
            self.tab_stats,
            "STATISTIQUES DESCRIPTIVES",
            "Analyses détaillées - Numériques et Catégorielles"
        )
        
        config_frame = ttk.Labelframe(
            self.tab_stats,
            text="Configuration",
            bootstyle="info",
            padding=15
        )
        config_frame.pack(fill=X, padx=20, pady=(20, 20))
        
        row1 = ttk.Frame(config_frame)
        row1.pack(fill=X, pady=5)
        
        ttk.Label(row1, text="Variable:", font=("Segoe UI", 10)).pack(side=LEFT, padx=5)
        
        self.stats_var_combo = ttk.Combobox(row1, width=30, state='readonly')
        self.stats_var_combo.pack(side=LEFT, padx=10)
        
        ttk.Button(
            row1,
            text="Calculer",
            command=self._calculate_stats,
            bootstyle="primary"
        ).pack(side=LEFT, padx=10)
        
        ttk.Button(
            row1,
            text="Toutes",
            command=self._calculate_all_stats,
            bootstyle="success-outline"
        ).pack(side=LEFT)
        
        results_frame = ttk.Labelframe(
            self.tab_stats,
            text="Résultats",
            bootstyle="primary",
            padding=15
        )
        results_frame.pack(fill=BOTH, expand=YES, padx=20, pady=(0, 20))
        
        scrollbar = ttk.Scrollbar(results_frame, bootstyle="primary-round")
        scrollbar.pack(side=RIGHT, fill=Y)
        
        self.stats_text = tk.Text(
            results_frame,
            wrap=WORD,
            font=("Consolas", 9),
            yscrollcommand=scrollbar.set,
            relief=FLAT,
            bg='#f8f9fa',
            fg='#212529',
            padx=15,
            pady=15
        )
        self.stats_text.pack(fill=BOTH, expand=YES)
        scrollbar.config(command=self.stats_text.yview)
        
        welcome = """
           STATISTIQUES DESCRIPTIVES

Sélectionnez une variable et cliquez sur "Calculer".
Supporte les variables numériques ET catégorielles.
"""
        self.stats_text.insert('1.0', welcome)
        self.stats_text.config(state=DISABLED)
    
    # ============================================================
    # ONGLET 4: VISUALISATIONS
    # ============================================================
    
    
    def _create_viz_tab(self):
        """Onglet Visualisations avec scroll sur le graphique"""
        
        # Header stylisé
        self._create_styled_header(
            self.tab_viz,
            "VISUALISATIONS",
            "Graphiques interactifs - Exploration visuelle"
        )
        
        controls = ttk.Labelframe(
            self.tab_viz,
            text="Configuration",
            bootstyle="info",
            padding=15
        )
        controls.pack(fill=X, padx=20, pady=(20, 10))
        
        row1 = ttk.Frame(controls)
        row1.pack(fill=X, pady=5)
        
        ttk.Label(row1, text="Type:", width=15).pack(side=LEFT, padx=5)
        
        self.viz_type = ttk.Combobox(
            row1,
            values=["Histogramme", "Boxplot", "Nuage de points", "Matrice de corrélation"],
            state='readonly',
            width=25
        )
        self.viz_type.pack(side=LEFT, padx=10)
        self.viz_type.current(0)
        
        row2 = ttk.Frame(controls)
        row2.pack(fill=X, pady=5)
        
        ttk.Label(row2, text="Variable X:", width=15).pack(side=LEFT, padx=5)
        self.viz_var1 = ttk.Combobox(row2, width=20, state='readonly')
        self.viz_var1.pack(side=LEFT, padx=10)
        
        ttk.Label(row2, text="Variable Y:", width=15).pack(side=LEFT, padx=5)
        self.viz_var2 = ttk.Combobox(row2, width=20, state='readonly')
        self.viz_var2.pack(side=LEFT, padx=10)
        
        ttk.Button(
            controls,
            text="Générer",
            command=self._generate_plot,
            bootstyle="success",
            width=30
        ).pack(pady=10)
        
        # Contrôles de zoom
        zoom_controls = ttk.Frame(controls)
        zoom_controls.pack(fill=X, pady=5)
        
        ttk.Label(zoom_controls, text="Zoom graphique:", font=("Segoe UI", 10, "bold")).pack(side=LEFT, padx=5)
        
        self.viz_zoom_label = ttk.Label(zoom_controls, text="100%", bootstyle="info")
        self.viz_zoom_label.pack(side=RIGHT, padx=5)
        
        ttk.Button(
            zoom_controls,
            text="−",
            command=self._viz_zoom_out,
            bootstyle="secondary",
            width=3
        ).pack(side=RIGHT, padx=2)
        
        ttk.Button(
            zoom_controls,
            text="+",
            command=self._viz_zoom_in,
            bootstyle="secondary",
            width=3
        ).pack(side=RIGHT, padx=2)
        
        ttk.Button(
            zoom_controls,
            text="Reset",
            command=self._viz_zoom_reset,
            bootstyle="info-outline",
            width=6
        ).pack(side=RIGHT, padx=5)
        
        # SOLUTION : Container avec scrollbars pour le graphique uniquement
        plot_container = ttk.Frame(self.tab_viz)
        plot_container.pack(fill=BOTH, expand=YES, padx=20, pady=(0, 20))
        
        # Canvas scrollable
        self.plot_canvas = tk.Canvas(plot_container, bg='white', highlightthickness=0)
        
        # Scrollbars
        v_scrollbar = ttk.Scrollbar(plot_container, orient=VERTICAL, command=self.plot_canvas.yview, bootstyle="primary-round")
        v_scrollbar.pack(side=RIGHT, fill=Y)
        
        h_scrollbar = ttk.Scrollbar(plot_container, orient=HORIZONTAL, command=self.plot_canvas.xview, bootstyle="primary-round")
        h_scrollbar.pack(side=BOTTOM, fill=X)
        
        self.plot_canvas.pack(side=LEFT, fill=BOTH, expand=YES)
        self.plot_canvas.configure(yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)
        
        # Frame pour le graphique DANS le canvas
        self.plot_frame = ttk.Frame(self.plot_canvas, style='Card.TFrame')
        self.plot_canvas_window = self.plot_canvas.create_window((0, 0), window=self.plot_frame, anchor="nw")
        
        # Fonction pour mettre à jour la zone scrollable
        def configure_plot_scroll(event=None):
            self.plot_canvas.configure(scrollregion=self.plot_canvas.bbox("all"))
        
        self.plot_frame.bind("<Configure>", configure_plot_scroll)
        
        # Adapter la largeur du frame au canvas
        def on_canvas_configure(event):
            canvas_width = event.width
            self.plot_canvas.itemconfig(self.plot_canvas_window, width=canvas_width)
        
        self.plot_canvas.bind("<Configure>", on_canvas_configure)

    def _viz_zoom_in(self):
        """Zoom avant"""
        self.viz_zoom_factor = min(3.0, self.viz_zoom_factor + 0.2)
        self._update_viz_zoom()

    def _viz_zoom_out(self):
        """Zoom arrière"""
        self.viz_zoom_factor = max(0.5, self.viz_zoom_factor - 0.2)
        self._update_viz_zoom()

    def _viz_zoom_reset(self):
        """Reset zoom"""
        self.viz_zoom_factor = 1.0
        self._update_viz_zoom()

    def _update_viz_zoom(self):
        """Appliquer zoom"""
        self.viz_zoom_label.config(text=f"{int(self.viz_zoom_factor * 100)}%")
        if self.current_fig:
            self._generate_plot()

    def _generate_plot(self):
        """Générer graphique"""
        if self.data is None:
            messagebox.showwarning("Attention", "Veuillez d'abord charger des données")
            return
        
        viz_type = self.viz_type.get()
        
        # Nettoyer
        for widget in self.plot_frame.winfo_children():
            widget.destroy()
        
        # Taille avec zoom - en pixels pour être sûr
        dpi = 100
        fig_width_inches = 10 * self.viz_zoom_factor
        fig_height_inches = 6 * self.viz_zoom_factor
        
        # Créer la figure
        fig = Figure(figsize=(fig_width_inches, fig_height_inches), facecolor='white', dpi=dpi)
        self.current_fig = fig
        ax = fig.add_subplot(111)
        
        try:
            if viz_type == "Histogramme":
                var = self.viz_var1.get()
                if var and var in self.numeric_vars:
                    ax.hist(self.data[var].dropna(), bins=30, color='#6366f1', alpha=0.7, edgecolor='black')
                    ax.set_title(f'Histogramme - {var}', fontsize=14, fontweight='bold')
                    ax.set_xlabel(var, fontsize=11)
                    ax.set_ylabel('Fréquence', fontsize=11)
                    ax.grid(True, alpha=0.3)
                else:
                    ax.text(0.5, 0.5, '⚠ Sélectionnez une variable numérique', 
                           ha='center', va='center', fontsize=14, transform=ax.transAxes)
                    ax.axis('off')
            
            elif viz_type == "Boxplot":
                var = self.viz_var1.get()
                if var and var in self.numeric_vars:
                    bp = ax.boxplot(self.data[var].dropna(), vert=True, patch_artist=True)
                    for patch in bp['boxes']:
                        patch.set_facecolor("#4A90E2")
                    ax.set_title(f'Boxplot - {var}', fontsize=14, fontweight='bold')
                    ax.set_ylabel(var, fontsize=11)
                    ax.grid(True, alpha=0.3, axis='y')
                else:
                    ax.text(0.5, 0.5, '⚠ Sélectionnez une variable numérique', 
                           ha='center', va='center', fontsize=14, transform=ax.transAxes)
                    ax.axis('off')
            
            elif viz_type == "Nuage de points":
                var1 = self.viz_var1.get()
                var2 = self.viz_var2.get()
                if var1 and var2 and var1 in self.numeric_vars and var2 in self.numeric_vars:
                    ax.scatter(self.data[var1], self.data[var2], c='#06b6d4', alpha=0.6, s=50)
                    ax.set_xlabel(var1, fontsize=11)
                    ax.set_ylabel(var2, fontsize=11)
                    ax.set_title(f'{var1} vs {var2}', fontsize=14, fontweight='bold')
                    ax.grid(True, alpha=0.3)
                else:
                    ax.text(0.5, 0.5, '⚠ Sélectionnez deux variables numériques', 
                           ha='center', va='center', fontsize=14, transform=ax.transAxes)
                    ax.axis('off')
            
            elif viz_type == "Matrice de corrélation":
                if self.numeric_vars and len(self.numeric_vars) >= 2:
                    vars_to_use = self.numeric_vars[:10]
                    corr = self.data[vars_to_use].corr()
                    
                    cax = ax.matshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
                    fig.colorbar(cax, ax=ax)
                    
                    ax.set_xticks(range(len(corr.columns)))
                    ax.set_yticks(range(len(corr.columns)))
                    ax.set_xticklabels(corr.columns, rotation=90, ha='left', fontsize=9)
                    ax.set_yticklabels(corr.columns, fontsize=9)
                    ax.set_title('Matrice de corrélation', fontsize=14, fontweight='bold', pad=20)
                else:
                    ax.text(0.5, 0.5, '⚠ Au moins 2 variables numériques requises', 
                           ha='center', va='center', fontsize=14, transform=ax.transAxes)
                    ax.axis('off')
            
            # Affichage avec tight_layout
            fig.tight_layout()
            
            # Créer le canvas matplotlib
            canvas = FigureCanvasTkAgg(fig, master=self.plot_frame)
            canvas.draw()
            
            # Obtenir le widget et le packager
            canvas_widget = canvas.get_tk_widget()
            canvas_widget.pack(fill=BOTH, expand=YES)
            
            # Mettre à jour la zone scrollable après ajout du graphique
            self.plot_frame.update_idletasks()
            self.plot_canvas.configure(scrollregion=self.plot_canvas.bbox("all"))
            
            # Scroller en haut
            self.plot_canvas.yview_moveto(0)
            
            print(f"✓ Graphique généré : {viz_type} ({fig_width_inches}x{fig_height_inches} inches, {dpi} dpi)")
            
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur:\n{str(e)}")
            print(f"✗ Erreur : {e}")
            import traceback
            traceback.print_exc()      
    
        # ============================================================
        # ONGLET 5: DONNÉES
        # ============================================================
        
    def _create_data_tab(self):
        """Onglet Données"""
            
        # Header stylisé
        self._create_styled_header(
            self.tab_data,
            "DONNÉES COMPLÈTES",
            "Vue tabulaire - Toutes les lignes"
        )
        
        tree_frame = ttk.Frame(self.tab_data)
        tree_frame.pack(fill=BOTH, expand=YES, padx=20, pady=(20, 20))
        
        vsb = ttk.Scrollbar(tree_frame, orient=VERTICAL, bootstyle="primary-round")
        vsb.pack(side=RIGHT, fill=Y)
        
        hsb = ttk.Scrollbar(tree_frame, orient=HORIZONTAL, bootstyle="primary-round")
        hsb.pack(side=BOTTOM, fill=X)
        
        self.tree = ttk.Treeview(
            tree_frame,
            yscrollcommand=vsb.set,
            xscrollcommand=hsb.set,
            bootstyle="primary"
        )
        self.tree.pack(fill=BOTH, expand=YES)
        
        vsb.config(command=self.tree.yview)
        hsb.config(command=self.tree.xview)
    
        # ============================================================
        # ONGLET 6: HISTORIQUE
        # ============================================================
        
    def _create_history_tab(self):
        """Onglet Historique"""
        
        # Header stylisé
        self._create_styled_header(
            self.tab_history,
            "HISTORIQUE",
            "Fichiers précédemment analysés"
        )
        
        btn_frame = ttk.Frame(self.tab_history)
        btn_frame.pack(fill=X, padx=20, pady=(20, 10))
        
        ttk.Button(
            btn_frame,
            text="Actualiser",
            command=self._load_history,
            bootstyle="info-outline"
        ).pack(side=LEFT, padx=5)
        
        ttk.Button(
            btn_frame,
            text="Effacer",
            command=self._clear_history,
            bootstyle="danger-outline"
        ).pack(side=LEFT)
        
        tree_frame = ttk.Frame(self.tab_history)
        tree_frame.pack(fill=BOTH, expand=YES, padx=20, pady=(0, 20))
        
        vsb = ttk.Scrollbar(tree_frame, orient=VERTICAL, bootstyle="primary-round")
        vsb.pack(side=RIGHT, fill=Y)
        
        columns = ('ID', 'Fichier', 'Date', 'Lignes', 'Colonnes', 'Score')
        self.history_tree = ttk.Treeview(
            tree_frame,
            columns=columns,
            show='headings',
            yscrollcommand=vsb.set,
            bootstyle="primary"
        )
        
        for col in columns:
            self.history_tree.heading(col, text=col)
            self.history_tree.column(col, width=100)
        
        self.history_tree.pack(fill=BOTH, expand=YES)
        vsb.config(command=self.history_tree.yview)
        
        self._load_history()
        
    # ============================================================
    # FONCTIONS UTILITAIRES
    # ============================================================
    
    def _detect_csv_separator(self, filepath: str) -> str:
        """Détecter séparateur CSV"""
        with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
            sample = ''.join([f.readline() for _ in range(5)])
        
        try:
            sniffer = csv.Sniffer()
            dialect = sniffer.sniff(sample, delimiters=',;\t|')
            return dialect.delimiter
        except:
            separators = [',', ';', '\t', '|', ' ']
            counts = {}
            
            for sep in separators:
                lines = sample.split('\n')
                if len(lines) > 1:
                    col_counts = [len(line.split(sep)) for line in lines[:3] if line.strip()]
                    if len(set(col_counts)) == 1 and col_counts[0] > 1:
                        counts[sep] = col_counts[0]
            
            if counts:
                return max(counts, key=counts.get)
            else:
                return ','
    
    def _open_file(self):
        """Ouvrir fichier CSV"""
        filename = filedialog.askopenfilename(
            title="Sélectionner un fichier CSV",
            filetypes=[("Fichiers CSV", "*.csv"), ("Tous", "*.*")]
        )
        
        if filename:
            try:
                separator = self._detect_csv_separator(filename)
                self.data = pd.read_csv(filename, sep=separator)
                
                if len(self.data.columns) == 1:
                    for sep in [';', '\t', '|', ' ']:
                        if sep != separator:
                            try:
                                test_data = pd.read_csv(filename, sep=sep)
                                if len(test_data.columns) > 1:
                                    self.data = test_data
                                    separator = sep
                                    break
                            except:
                                continue
                
                self.filename = os.path.basename(filename)
                self.filepath = filename
                
                sep_name = {
                    ',': 'virgule',
                    ';': 'point-virgule',
                    '\t': 'tabulation',
                    '|': 'barre',
                    ' ': 'espace'
                }.get(separator, separator)
                
                self._detect_variable_types()
                self._update_ui_after_load()
                self._add_to_history()
                self._update_results_tab_info()
                
                ToastNotification(
                    title="Succès",
                    message=f"{self.data.shape[0]:,} lignes - Sep: {sep_name}",
                    duration=3000,
                    bootstyle="success"
                ).show_toast()
                
            except Exception as e:
                messagebox.showerror("Erreur", f"Erreur:\n{str(e)}")
    
    def _open_excel_file(self):
        """Ouvrir fichier Excel"""
        filename = filedialog.askopenfilename(
            title="Sélectionner un fichier Excel",
            filetypes=[("Fichiers Excel", "*.xlsx *.xls"), ("Tous", "*.*")]
        )
        
        if filename:
            try:
                # Lire le fichier Excel
                self.data = pd.read_excel(filename, engine='openpyxl')
                
                self.filename = os.path.basename(filename)
                self.filepath = filename
                
                self._detect_variable_types()
                self._update_ui_after_load()
                self._add_to_history()
                self._update_results_tab_info()
                
                ToastNotification(
                    title="Succès",
                    message=f"{self.data.shape[0]:,} lignes chargées depuis Excel",
                    duration=3000,
                    bootstyle="success"
                ).show_toast()
                
            except Exception as e:
                messagebox.showerror("Erreur", f"Erreur lors de l'ouverture du fichier Excel:\n{str(e)}")
    
    def _detect_variable_types(self):
        """Détecter types"""
        self.variable_types = {}
        self.numeric_vars = []
        self.categorical_vars = []
        self.boolean_vars = []
        
        for col in self.data.columns:
            dtype = self.data[col].dtype
            unique_count = self.data[col].nunique()
            total_count = len(self.data[col])
            
            if unique_count == 2 and self.data[col].dropna().isin([0, 1, True, False, 'True', 'False', 'yes', 'no']).all():
                self.variable_types[col] = 'Booléenne'
                self.boolean_vars.append(col)
            elif dtype in ['int64', 'float64', 'int32', 'float32']:
                self.variable_types[col] = 'Numérique'
                self.numeric_vars.append(col)
            elif dtype == 'object' or unique_count < total_count * 0.05:
                self.variable_types[col] = 'Catégorielle'
                self.categorical_vars.append(col)
    
    def _update_ui_after_load(self):
        """MAJ UI"""
        self.lbl_filename.config(text=self.filename)
        self.lbl_dimensions.config(text=f"{self.data.shape[0]:,} × {self.data.shape[1]}")
        
        self._display_data_preview()
        
        self.var1_combo['values'] = self.numeric_vars
        self.var2_combo['values'] = self.numeric_vars
        self.cat_combo['values'] = self.categorical_vars + self.boolean_vars
        
        if self.numeric_vars:
            self.var1_combo.current(0)
            if len(self.numeric_vars) > 1:
                self.var2_combo.current(1)
        
        if self.categorical_vars:
            self.cat_combo.current(0)
        
        # Stats - TOUTES les variables
        all_vars = self.numeric_vars + self.categorical_vars + self.boolean_vars
        self.stats_var_combo['values'] = all_vars
        if all_vars:
            self.stats_var_combo.current(0)
        
        self.viz_var1['values'] = self.numeric_vars
        self.viz_var2['values'] = self.numeric_vars
        if self.numeric_vars:
            self.viz_var1.current(0)
            if len(self.numeric_vars) > 1:
                self.viz_var2.current(1)
        
        self._display_data_in_tree()
    
    def _display_data_preview(self):
        """Aperçu données"""
        self.data_preview.config(state=NORMAL)
        self.data_preview.delete('1.0', END)
        
        n_rows = min(50, len(self.data))
        preview_data = self.data.head(n_rows).to_string()
        
        header = f"""
  APERÇU - {n_rows} lignes sur {len(self.data):,}

"""
        self.data_preview.insert('1.0', header + preview_data)
        self.data_preview.config(state=DISABLED)
    
    def _display_data_in_tree(self):
        """Afficher dans treeview"""
        self.tree.delete(*self.tree.get_children())
        
        self.tree['columns'] = list(self.data.columns)
        self.tree['show'] = 'headings'
        
        for col in self.data.columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)
        
        for idx, row in self.data.head(100).iterrows():
            self.tree.insert('', END, values=list(row))
    
    def _add_to_history(self):
        """Ajouter historique"""
        data_info = {
            'filename': self.filename,
            'filepath': self.filepath,
            'rows': len(self.data),
            'columns': len(self.data.columns),
            'numeric_vars': len(self.numeric_vars),
            'categorical_vars': len(self.categorical_vars),
            'boolean_vars': len(self.boolean_vars),
            'quality_score': 0,
            'missing_pct': 0,
            'outliers_count': 0
        }
        
        self.history_manager.add_entry(data_info)
        self._load_history()
    
    def _load_history(self):
        """Charger historique"""
        for item in self.history_tree.get_children():
            self.history_tree.delete(item)
        
        history = self.history_manager.get_history()
        
        for entry in history:
            self.history_tree.insert('', END, values=(
                entry['id'],
                entry['filename'],
                entry['loaded_at'][:19],
                f"{entry['rows']:,}",
                entry['columns'],
                f"{entry.get('quality_score', 0):.1f}/100"
            ))
    
    def _clear_history(self):
        """Effacer historique"""
        if Messagebox.yesno("Confirmation", "Effacer l'historique ?"):
            if os.path.exists("eda_history.db"):
                os.remove("eda_history.db")
            
            self.history_manager = HistoryManager()
            self._load_history()
            
            ToastNotification(
                title="OK",
                message="Historique effacé",
                duration=2000,
                bootstyle="success"
            ).show_toast()
    
    def _go_to_stats(self):
        """Aller stats"""
        if self.data is None:
            messagebox.showwarning("Attention", "Aucun fichier")
            return
        
        var = self.var1_combo.get()
        if var:
            self.stats_var_combo.set(var)
        
        self.notebook.select(2)
        
        ToastNotification(
            title="Navigation",
            message="Statistiques",
            duration=2000,
            bootstyle="info"
        ).show_toast()
    
    # ============================================================
    # ANALYSES - AVEC ACCUMULATION
    # ============================================================
    
    def _show_detailed_types(self):
        """Types détaillés"""
        if self.data is None:
            messagebox.showwarning("Attention", "Aucun fichier")
            return
        
        report = """
                    DÉTECTION AUTOMATIQUE DES TYPES

"""
        
        if self.numeric_vars:
            report += "\n VARIABLES NUMÉRIQUES\n" + "-" * 80 + "\n"
            for i, var in enumerate(self.numeric_vars, 1):
                dtype = self.data[var].dtype
                n_unique = self.data[var].nunique()
                min_val = self.data[var].min()
                max_val = self.data[var].max()
                report += f"{i:2d}. {var:30s} | {str(dtype):10s} | {n_unique:6d} valeurs | [{min_val:.2f}, {max_val:.2f}]\n"
        
        if self.categorical_vars:
            report += "\n\n VARIABLES CATÉGORIELLES\n" + "-" * 80 + "\n"
            for i, var in enumerate(self.categorical_vars, 1):
                n_unique = self.data[var].nunique()
                report += f"{i:2d}. {var:30s} | {n_unique:6d} modalités\n"
        
        if self.boolean_vars:
            report += "\n\n VARIABLES BOOLÉENNES\n" + "-" * 80 + "\n"
            for i, var in enumerate(self.boolean_vars, 1):
                report += f"{i:2d}. {var:30s}\n"
        
        report += f"""

                                  RÉCAPITULATIF

    Total : {len(self.data.columns)}
    
    Numériques    : {len(self.numeric_vars):3d}
    Catégorielles : {len(self.categorical_vars):3d}
    Booléennes    : {len(self.boolean_vars):3d}
"""
        
        # Zone 4 (aperçu)
        self.results_text.config(state=NORMAL)
        self.results_text.delete('1.0', END)
        self.results_text.insert('1.0', report)
        self.results_text.config(state=DISABLED)
        self.status_badge.config(text="Terminé")
        
        # ACCUMULER
        self._add_analysis_to_accumulator("Types de variables", report)
        
        ToastNotification(
            title="Ajouté",
            message="Voir l'onglet Résultats",
            duration=2000,
            bootstyle="success"
        ).show_toast()
    
    def _analyze_missing_values(self):
        """Valeurs manquantes"""
        if self.data is None:
            messagebox.showwarning("Attention", "Aucun fichier")
            return
        
        self.missing_values = {}
        self.high_missing_vars = []
        
        for col in self.data.columns:
            missing_count = self.data[col].isnull().sum()
            missing_pct = (missing_count / len(self.data)) * 100
            self.missing_values[col] = (missing_count, missing_pct)
            if missing_pct > 30:
                self.high_missing_vars.append(col)
        
        self._display_missing_report()
    
    def _display_missing_report(self):
        """Rapport missing"""
        total_missing = sum(count for count, _ in self.missing_values.values())
        total_cells = len(self.data) * len(self.data.columns)
        
        report = f"""
                         VALEURS MANQUANTES

Total : {total_missing:,} / {total_cells:,} ({total_missing/total_cells*100:.2f}%)

"""
        
        vars_with_missing = [(col, count, pct) for col, (count, pct) in self.missing_values.items() if count > 0]
        vars_with_missing.sort(key=lambda x: x[2], reverse=True)
        
        if vars_with_missing:
            report += "Variables concernées:\n" + "-" * 80 + "\n"
            for col, count, pct in vars_with_missing[:15]:
                level = "CRITIQUE" if pct > 50 else "ELEVE" if pct > 30 else "MOYEN" if pct > 10 else "FAIBLE"
                report += f"[{level}] {col[:35]:<35} {count:>8,} ({pct:>5.1f}%)\n"
        else:
            report += " Aucune valeur manquante\n"
        
        self.results_text.config(state=NORMAL)
        self.results_text.delete('1.0', END)
        self.results_text.insert('1.0', report)
        self.results_text.config(state=DISABLED)
        self.status_badge.config(text="Terminé")
        
        # ACCUMULER
        self._add_analysis_to_accumulator("Valeurs manquantes", report)
        
        ToastNotification(
            title="Ajouté",
            message="Voir l'onglet Résultats",
            duration=2000,
            bootstyle="success"
        ).show_toast()
    
    def _detect_quasi_constant(self):
        """Variables constantes"""
        if self.data is None:
            return
        
        self.quasi_constant_vars = []
        for col in self.data.columns:
            value_counts = self.data[col].value_counts(dropna=False)
            if len(value_counts) > 0:
                if value_counts.iloc[0] / len(self.data) > 0.95:
                    self.quasi_constant_vars.append(col)
        
        self._display_constant_report()
    
    def _display_constant_report(self):
        """Rapport constantes"""
        report = f"""
                      VARIABLES QUASI-CONSTANTES

Détectées : {len(self.quasi_constant_vars)}

"""
        
        for var in self.quasi_constant_vars:
            report += f"- {var}\n"
        
        if not self.quasi_constant_vars:
            report += " Aucune variable quasi-constante\n"
        
        self.results_text.config(state=NORMAL)
        self.results_text.delete('1.0', END)
        self.results_text.insert('1.0', report)
        self.results_text.config(state=DISABLED)
        self.status_badge.config(text="Terminé")
        
        # ACCUMULER
        self._add_analysis_to_accumulator("Variables constantes", report)
        
        ToastNotification(
            title="Ajouté",
            message="Voir l'onglet Résultats",
            duration=2000,
            bootstyle="success"
        ).show_toast()
    
    def _detect_outliers(self):
        """Outliers"""
        if self.data is None or not self.numeric_vars:
            return
        
        self.outliers_info = {}
        for col in self.numeric_vars:
            Q1 = self.data[col].quantile(0.25)
            Q3 = self.data[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            
            outliers_mask = (self.data[col] < lower) | (self.data[col] > upper)
            count = outliers_mask.sum()
            
            if count > 0:
                self.outliers_info[col] = {
                    'count': count,
                    'percentage': (count / len(self.data)) * 100
                }
        
        self._display_outliers_report()
    
    def _display_outliers_report(self):
        """Rapport outliers"""
        report = f"""
                       OUTLIERS (MÉTHODE IQR)

Variables avec outliers : {len(self.outliers_info)}

"""
        
        for col, info in sorted(self.outliers_info.items(), key=lambda x: x[1]['percentage'], reverse=True):
            level = "ELEVE" if info['percentage'] > 10 else "MOYEN" if info['percentage'] > 5 else "FAIBLE"
            report += f"[{level}] {col[:35]:<35} {info['count']:>6,} ({info['percentage']:>5.1f}%)\n"
        
        if not self.outliers_info:
            report += " Aucun outlier significatif\n"
        
        self.results_text.config(state=NORMAL)
        self.results_text.delete('1.0', END)
        self.results_text.insert('1.0', report)
        self.results_text.config(state=DISABLED)
        self.status_badge.config(text="Terminé")
        
        # ACCUMULER
        self._add_analysis_to_accumulator("Outliers (IQR)", report)
        
        ToastNotification(
            title="Ajouté",
            message="Voir l'onglet Résultats",
            duration=2000,
            bootstyle="success"
        ).show_toast()
    
    def _full_quality_report(self):
        """Rapport complet"""
        if self.data is None:
            return
        
        self._analyze_missing_values()
        self._detect_quasi_constant()
        self._detect_outliers()
        
        quality_score = self._calculate_quality_score()
        
        total_missing = sum(count for count, _ in self.missing_values.values())
        total_cells = len(self.data) * len(self.data.columns)
        
        if quality_score >= 90:
            grade = "EXCELLENT"
        elif quality_score >= 75:
            grade = "BON"
        elif quality_score >= 60:
            grade = "MOYEN"
        else:
            grade = "FAIBLE"
        
        report = f"""
                         RAPPORT COMPLET DE QUALITÉ

INFORMATIONS
{'-' * 85}
Fichier      : {self.filename}
Dimensions   : {self.data.shape[0]:,} × {self.data.shape[1]}
Mémoire      : {self.data.memory_usage(deep=True).sum() / 1024**2:.2f} MB

SCORE : {quality_score:.1f}/100  {grade}

DIAGNOSTIC
{'-' * 85}
Valeurs manquantes    : {total_missing:,} ({total_missing/total_cells*100:.2f}%)
Variables constantes  : {len(self.quasi_constant_vars)}
Variables avec outliers: {len(self.outliers_info)}
"""
        
        self.results_text.config(state=NORMAL)
        self.results_text.delete('1.0', END)
        self.results_text.insert('1.0', report)
        self.results_text.config(state=DISABLED)
        self.status_badge.config(text="Rapport généré")
        
        # ACCUMULER
        self._add_analysis_to_accumulator("Rapport complet de qualité", report)
        
        # Basculer vers Résultats
        self.notebook.select(1)
        
        ToastNotification(
            title="Rapport complet",
            message=f"{len(self.accumulated_reports)} analyse(s) disponible(s)",
            duration=3000,
            bootstyle="success"
        ).show_toast()
    
    def _calculate_quality_score(self):
        """Score qualité"""
        score = 0
        
        if self.missing_values:
            total_missing = sum(count for count, _ in self.missing_values.values())
            total_cells = len(self.data) * len(self.data.columns)
            completeness = 1 - (total_missing / total_cells)
            score += completeness * 40
        else:
            score += 40
        
        if len(self.data.columns) > 0:
            ratio_good = 1 - (len(self.quasi_constant_vars) / len(self.data.columns))
            score += ratio_good * 30
        else:
            score += 30
        
        if len(self.numeric_vars) > 0 and self.outliers_info:
            avg_outliers = sum(info['percentage'] for info in self.outliers_info.values()) / len(self.numeric_vars)
            score += max(0, 30 - avg_outliers * 2)
        else:
            score += 30
        
        return score
    
    # ============================================================
    # STATISTIQUES - AVEC SUPPORT CATÉGORIELLES
    # ============================================================
    
    def _calculate_stats(self):
        """Calculer stats (numérique OU catégorielle)"""
        if self.data is None:
            return
        
        var = self.stats_var_combo.get()
        if not var:
            return
        
        self.stats_text.config(state=NORMAL)
        self.stats_text.delete('1.0', END)
        
        if var in self.numeric_vars:
            # NUMÉRIQUE
            data_var = self.data[var].dropna()
            
            self.current_stats[var] = {
                'Moyenne': data_var.mean(),
                'Médiane': data_var.median(),
                'Écart-type': data_var.std(),
                'Variance': data_var.var(),
                'Min': data_var.min(),
                'Max': data_var.max(),
                'Q1': data_var.quantile(0.25),
                'Q3': data_var.quantile(0.75),
                'Skewness': data_var.skew(),
                'Kurtosis': data_var.kurtosis()
            }
            
            report = f"""
      STATISTIQUES NUMÉRIQUES - {var[:40]:<40}

"""
            for stat, value in self.current_stats[var].items():
                report += f"{stat:<20} : {value:>12.4f}\n"
        
        else:
            # CATÉGORIELLE
            value_counts = self.data[var].value_counts(dropna=False)
            value_percentages = self.data[var].value_counts(normalize=True, dropna=False) * 100
            
            n_total = len(self.data[var])
            n_missing = self.data[var].isnull().sum()
            n_unique = self.data[var].nunique()
            
            report = f"""
    STATISTIQUES CATÉGORIELLES - {var[:40]:<40}

INFORMATIONS
{'-' * 70}
Total           : {n_total:,}
Manquantes      : {n_missing:,} ({n_missing/n_total*100:.2f}%)
Valeurs uniques : {n_unique:,}

MODALITÉS DOMINANTES (Top 10)
{'-' * 70}
{'Modalité':<30} {'Effectif':>12} {'%':>12}
{'-' * 70}
"""
            
            for i, (modalite, count) in enumerate(value_counts.head(10).items()):
                pct = value_percentages[modalite]
                modalite_str = str(modalite)[:28]
                report += f"{modalite_str:<30} {count:>12,} {pct:>11.2f}%\n"
            
            if len(value_counts) > 10:
                report += f"\n... et {len(value_counts) - 10} autres modalités\n"
        
        self.stats_text.insert('1.0', report)
        self.stats_text.config(state=DISABLED)
    
    def _calculate_all_stats(self):
        """Stats globales"""
        if self.data is None:
            return
        
        self.stats_text.config(state=NORMAL)
        self.stats_text.delete('1.0', END)
        
        report = """
         STATISTIQUES GLOBALES

"""
        
        # Numériques
        if self.numeric_vars:
            report += f"\nVARIABLES NUMÉRIQUES\n{'=' * 70}\n"
            for var in self.numeric_vars[:10]:
                data_var = self.data[var].dropna()
                report += f"\n{var}\n{'-' * 70}\n"
                report += f"Moy: {data_var.mean():.2f} | Med: {data_var.median():.2f} | Std: {data_var.std():.2f}\n"
                report += f"Min: {data_var.min():.2f} | Max: {data_var.max():.2f}\n"
        
        # Catégorielles
        if self.categorical_vars:
            report += f"\n\nVARIABLES CATÉGORIELLES\n{'=' * 70}\n"
            
            for var in self.categorical_vars[:10]:
                value_counts = self.data[var].value_counts()
                n_unique = self.data[var].nunique()
                
                report += f"\n{var} ({n_unique} modalités)\n{'-' * 70}\n"
                
                for i, (modalite, count) in enumerate(value_counts.head(3).items()):
                    pct = (count / len(self.data[var])) * 100
                    report += f"  {i+1}. {str(modalite)[:30]:<30} : {count:>6,} ({pct:>5.1f}%)\n"
        
        self.stats_text.insert('1.0', report)
        self.stats_text.config(state=DISABLED)
    
    # ============================================================
    # VISUALISATIONS
    # ============================================================
    
    def _generate_plot(self):
        """Générer graphique"""
        if self.data is None:
            return
        
        viz_type = self.viz_type.get()
        
        for widget in self.plot_frame.winfo_children():
            widget.destroy()
        
        fig = Figure(figsize=(10, 6), facecolor='white')
        ax = fig.add_subplot(111)
        
        try:
            if viz_type == "Histogramme":
                var = self.viz_var1.get()
                if var:
                    ax.hist(self.data[var].dropna(), bins=30, color='#6366f1', alpha=0.7, edgecolor='black')
                    ax.set_title(f'Histogramme - {var}', fontsize=14, fontweight='bold')
                    ax.grid(True, alpha=0.3)
            
            elif viz_type == "Boxplot":
                var = self.viz_var1.get()
                if var:
                    bp = ax.boxplot(self.data[var].dropna(), vert=True, patch_artist=True)
                    for patch in bp['boxes']:
                        patch.set_facecolor("#4A90E2")
                    ax.set_title(f'Boxplot - {var}', fontsize=14, fontweight='bold')
                    ax.grid(True, alpha=0.3, axis='y')
            
            elif viz_type == "Nuage de points":
                var1, var2 = self.viz_var1.get(), self.viz_var2.get()
                if var1 and var2:
                    ax.scatter(self.data[var1], self.data[var2], c='#06b6d4', alpha=0.6, s=50)
                    ax.set_xlabel(var1)
                    ax.set_ylabel(var2)
                    ax.set_title(f'{var1} vs {var2}', fontsize=14, fontweight='bold')
                    ax.grid(True, alpha=0.3)
            
            elif viz_type == "Matrice de corrélation":
                if self.numeric_vars:
                    corr = self.data[self.numeric_vars[:10]].corr()
                    cax = ax.matshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
                    fig.colorbar(cax)
                    ax.set_xticks(range(len(corr.columns)))
                    ax.set_yticks(range(len(corr.columns)))
                    ax.set_xticklabels(corr.columns, rotation=90)
                    ax.set_yticklabels(corr.columns)
                    ax.set_title('Matrice de corrélation', fontsize=14, fontweight='bold', pad=20)
            
            canvas = FigureCanvasTkAgg(fig, master=self.plot_frame)
            canvas.draw()
            canvas.get_tk_widget().pack(fill=BOTH, expand=YES)
            
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur:\n{str(e)}")
    
    # ============================================================
    # EXPORTS
    # ============================================================
    
    def _export_word(self):
        """Export Word avec contenu accumulé complet"""
        if self.data is None:
            messagebox.showwarning("Attention", "Aucun fichier")
            return
        
        output_path = filedialog.asksaveasfilename(
            defaultextension=".docx",
            filetypes=[("Word", "*.docx")],
            initialfile=f"rapport_{self.filename.replace('.csv', '').replace('.xlsx', '')}.docx"
        )
        
        if output_path:
            try:
                quality_score = self._calculate_quality_score()
                
                total_missing = sum(count for count, _ in self.missing_values.values()) if self.missing_values else 0
                total_cells = len(self.data) * len(self.data.columns)
                
                data_info = {
                    'filename': self.filename,
                    'rows': len(self.data),
                    'columns': len(self.data.columns),
                    'numeric_vars': len(self.numeric_vars),
                    'categorical_vars': len(self.categorical_vars),
                    'boolean_vars': len(self.boolean_vars),
                    'quality_score': quality_score,
                    'missing_pct': (total_missing / total_cells * 100) if total_cells > 0 else 0,
                    'outliers_count': len(self.outliers_info),
                    'constant_vars': len(self.quasi_constant_vars),
                    'analyses_count': len(self.accumulated_reports)
                }
                
                # PASSER LE CONTENU ACCUMULÉ
                self.report_exporter.export_to_word(
                    data_info, 
                    self.current_stats, 
                    output_path,
                    accumulated_content=self.last_analysis_report
                )
                
                ToastNotification(
                    title="Succès",
                    message="Rapport Word complet généré",
                    duration=3000,
                    bootstyle="success"
                ).show_toast()
                
            except Exception as e:
                messagebox.showerror("Erreur", f"Erreur:\n{str(e)}")
    
    def _export_pdf(self):
        """Export PDF avec contenu accumulé complet"""
        if self.data is None:
            messagebox.showwarning("Attention", "Aucun fichier")
            return
        
        output_path = filedialog.asksaveasfilename(
            defaultextension=".pdf",
            filetypes=[("PDF", "*.pdf")],
            initialfile=f"rapport_{self.filename.replace('.csv', '').replace('.xlsx', '')}.pdf"
        )
        
        if output_path:
            try:
                quality_score = self._calculate_quality_score()
                
                total_missing = sum(count for count, _ in self.missing_values.values()) if self.missing_values else 0
                total_cells = len(self.data) * len(self.data.columns)
                
                data_info = {
                    'filename': self.filename,
                    'rows': len(self.data),
                    'columns': len(self.data.columns),
                    'numeric_vars': len(self.numeric_vars),
                    'categorical_vars': len(self.categorical_vars),
                    'boolean_vars': len(self.boolean_vars),
                    'quality_score': quality_score,
                    'missing_pct': (total_missing / total_cells * 100) if total_cells > 0 else 0,
                    'outliers_count': len(self.outliers_info),
                    'constant_vars': len(self.quasi_constant_vars),
                    'analyses_count': len(self.accumulated_reports)
                }
                
                # PASSER LE CONTENU ACCUMULÉ
                self.report_exporter.export_to_pdf(
                    data_info, 
                    self.current_stats, 
                    output_path,
                    accumulated_content=self.last_analysis_report
                )
                
                ToastNotification(
                    title="Succès",
                    message="Rapport PDF complet généré",
                    duration=3000,
                    bootstyle="success"
                ).show_toast()
                
            except Exception as e:
                messagebox.showerror("Erreur", f"Erreur:\n{str(e)}")
    
    def _export_excel(self):
        """Export Excel"""
        if self.data is None:
            messagebox.showwarning("Attention", "Aucun fichier")
            return
        
        output_path = filedialog.asksaveasfilename(
            defaultextension=".xlsx",
            filetypes=[("Excel", "*.xlsx")],
            initialfile=f"export_{self.filename.replace('.csv', '').replace('.xlsx', '')}.xlsx"
        )
        
        if output_path:
            try:
                with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                    self.data.to_excel(writer, sheet_name='Données', index=False)
                    
                    if self.current_stats:
                        stats_df = pd.DataFrame(self.current_stats)
                        stats_df.to_excel(writer, sheet_name='Statistiques')
                
                ToastNotification(
                    title="Succès",
                    message="Export Excel terminé",
                    duration=3000,
                    bootstyle="success"
                ).show_toast()
                
            except Exception as e:
                messagebox.showerror("Erreur", f"Erreur:\n{str(e)}")
    
    def _show_about(self):
        """À propos"""
        about_text = """EDA-Desk PRO HYBRID v4.0 ENHANCED

Fonctionnalités complètes:
- Architecture 4 zones respectée
- Support CSV et Excel (.xlsx)
- Zoom adaptatif (50% à 200%)
- Headers stylisés uniformes
- Accumulation progressive des résultats
- Statistiques numériques ET catégorielles
- Exports professionnels (Word, PDF, Excel)
- Visualisations interactives
- Historique complet
- Interface moderne avec couleurs

Raccourcis clavier:
- Ctrl+O : Ouvrir CSV
- Ctrl+E : Ouvrir Excel
- Ctrl++ : Zoom avant
- Ctrl+- : Zoom arrière
- Ctrl+0 : Réinitialiser zoom

Développé avec Python, tkinter & ttkbootstrap
"""
        Messagebox.show_info(about_text, "À propos")


def main():
    """Point d'entrée"""
    root = ttk.Window(themename="flatly")
    app = EDADeskHybrid(root)
    root.mainloop()


if __name__ == "__main__":
    main()

C:\Users\admin\AppData\Local\Temp\ipykernel_7160\168053637.py:1603: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(filename, sep=separator)
